[XGBoost]
- gradient boosting 라이브러리 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

< 데이터 불러오기 >

In [30]:
data_path = '../data/'

train_df = pd.read_csv(data_path+'train.csv', index_col='id')
test_df = pd.read_csv(data_path+'test.csv', index_col='id')
submission_df = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

< 데이터 분리>

In [23]:
# 피처와 타겟 변수 분리 
X = train_df.drop(columns=['defects'])
y = train_df['defects']

# 훈련 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



< XGBoost 모델 훈련>

In [26]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# 파라미터 설정 (분류 문제를 위한 기본 설정)
params = {
    'objective': 'binary:logistic',  # 이진 분류 문제
    'eval_metric': 'logloss',
    'eta': 0.01,
    'max_depth': 6
}

# 모델 훈련
num_rounds = 1000
watchlist = [(dtrain, 'train'), (dval, 'val')]
model = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=50)


[0]	train-logloss:0.53585	val-logloss:0.53028
[1]	train-logloss:0.53372	val-logloss:0.52825
[2]	train-logloss:0.53164	val-logloss:0.52628
[3]	train-logloss:0.52961	val-logloss:0.52434
[4]	train-logloss:0.52763	val-logloss:0.52246


c:\Python39\Lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[5]	train-logloss:0.52569	val-logloss:0.52061
[6]	train-logloss:0.52380	val-logloss:0.51880
[7]	train-logloss:0.52194	val-logloss:0.51704
[8]	train-logloss:0.52014	val-logloss:0.51532
[9]	train-logloss:0.51837	val-logloss:0.51364
[10]	train-logloss:0.51664	val-logloss:0.51199
[11]	train-logloss:0.51495	val-logloss:0.51039
[12]	train-logloss:0.51330	val-logloss:0.50881
[13]	train-logloss:0.51168	val-logloss:0.50727
[14]	train-logloss:0.51009	val-logloss:0.50577
[15]	train-logloss:0.50854	val-logloss:0.50430
[16]	train-logloss:0.50702	val-logloss:0.50286
[17]	train-logloss:0.50553	val-logloss:0.50145
[18]	train-logloss:0.50408	val-logloss:0.50007
[19]	train-logloss:0.50265	val-logloss:0.49872
[20]	train-logloss:0.50125	val-logloss:0.49740
[21]	train-logloss:0.49988	val-logloss:0.49611
[22]	train-logloss:0.49854	val-logloss:0.49484
[23]	train-logloss:0.49722	val-logloss:0.49360
[24]	train-logloss:0.49593	val-logloss:0.49238
[25]	train-logloss:0.49466	val-logloss:0.49119
[26]	train-logloss

< 성능 평가: Accuracy>

In [33]:
# 예측 (확률값으로 예측)
from sklearn.metrics import accuracy_score

y_pred_proba = model.predict(dval)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_proba]  # 확률값을 기준으로 이진 분류

# 성능 평가
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 결과해석: 정확도 0.8144 (81.44%)


Accuracy: 0.8146


< 성능 평가: ROC-AUC Score>

In [34]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',  # 여기서 'auc'를 지정하여 ROC-AUC를 평가 지표로 사용
    'eta': 0.01,
    'max_depth': 6
}

model = xgb.train(params, dtrain, num_boost_round=1000, 
                  evals=[(dtrain, 'train'), (dval, 'val')],
                  early_stopping_rounds=50, verbose_eval=20)

# 결과해석: 학습이 중단된 시점의 train-auc는 0.80505, val-auc는 0.78832
# 모델이 학습데이터에 대해 약 80.5%의 ROC-AUC 성능, 검증데이터에 대해서는 약 78.8%의 ROC-AUC 성능을 보여줌

[0]	train-auc:0.79278	val-auc:0.78274
[20]	train-auc:0.79420	val-auc:0.78519
[40]	train-auc:0.79596	val-auc:0.78658
[60]	train-auc:0.79670	val-auc:0.78660
[80]	train-auc:0.79765	val-auc:0.78660
[100]	train-auc:0.79845	val-auc:0.78720
[120]	train-auc:0.79948	val-auc:0.78789
[140]	train-auc:0.80033	val-auc:0.78818
[160]	train-auc:0.80106	val-auc:0.78809
[180]	train-auc:0.80171	val-auc:0.78824
[200]	train-auc:0.80245	val-auc:0.78821
[220]	train-auc:0.80312	val-auc:0.78836
[240]	train-auc:0.80383	val-auc:0.78827
[260]	train-auc:0.80451	val-auc:0.78825
[275]	train-auc:0.80505	val-auc:0.78832
